## Create large 2D phantom

This notebook creates a (relatively) large 2D phantom for testing different downsampling schemes.  Right now setup to compare integer sub-sampling with aggregation using mode, specifically looking at the effect of each technique on measurements with smaller labels.  We look at a couple of scenarios

1.  If many (or all) labels are a single pixel, perhaps representing events or measurements involving single molecules.
2.  We also look at the case of slightly bigger (but still small) labels of radius 2-10 pixels. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tnia.plotting.plt_helper import imshow_multi2d
from pathlib import Path


##  Create function to add random circles to an image 

If the radius is below 0.5 the circles will be a single point

In [2]:
from random import seed, uniform
import raster_geometry as rg
import math

def add_lots_of_circles(im, label, num, min_r, max_r, min_intensity, max_intensity, start_label=0, seed_val=-1):
    """ draws random circles in an image useful for testing segmentation, background subtraction etc.

    Args:
        im (2d numpy array): input image, will be changed
        num (number): number of circles to draw
        min_r (number): min radius of circles
        max_r (number): max radius of circles
        min_intensity (number): max intensity of circles
        max_intensity (number): min intensity of circles
        seed_val (int, optional): use seed if you need to replicate the same random image defalts to -1 (no seed used).
    """
    if seed_val!=-1:
        seed(seed_val)

    for i in range(num):
        label_num = start_label+i
        r=uniform(min_r, max_r)
        cx=round(uniform(r,im.shape[1]-r))
        cy=round(uniform(r,im.shape[0]-r))
        intensity=round(uniform(min_intensity, max_intensity))
        #print(r,cx,cy,intensity)
        size = [math.ceil(r**2), math.ceil(r**2)]
        temp=rg.circle(size,r)
        
        startx = cx-math.ceil(r)
        starty = cy-math.ceil(r)
        
        if startx<0:
            startx=0
        if starty<0:
            starty=0
        if startx+temp.shape[1]>im.shape[1]:
            startx = im.shape[1]-temp.shape[1]
        if starty+temp.shape[0]>im.shape[0]:
            starty = im.shape[0]-temp.shape[0]
        
        im[starty:starty+temp.shape[0],startx:startx+temp.shape[1]]=intensity*temp
        label[starty:starty+temp.shape[0],startx:startx+temp.shape[1]]=label_num*temp
        #im[temp2>0]=im[temp2>0]+intensity


## Generate phantoms for different scenarios

In [3]:
# scenarois 
# 1 all subresolution objects
# 2 type 1 subresoluition, type 2 a bit bigger
# 3 type 1 ranges from 2.5 to 4.5, type 2 from 4.5 to 8.5

scenario_names = ["all subresolution objects", "type 1 subresoluition, type 2 a bit bigger", "type 1 rangest from 2.5 to 4.5, type 2 from 4.5 to 8.5"]
scenario = 2

if scenario == 0:
    min_size1, max_size1 = 0.5, 0.5
    min_size2, max_size2 = 0.5, 0.5
    num1 = 2000
    num2 = 1000
elif scenario == 1:
    min_size1, max_size1 = 0.5, 0.5
    min_size2, max_size2 = 1.5, 2.5
    num1 = 2000
    num2 = 1000
elif scenario == 2:
    min_size1, max_size1 = 2.5, 4.5
    min_size2, max_size2 = 4.5, 8.5
    num1 = 200
    num2 = 100

phantom = np.zeros((10000, 10000), dtype=np.int8)
labels = np.zeros((10000, 10000), dtype=np.int16)
add_lots_of_circles(phantom, labels, num1, min_size1, max_size1, 1, 1, start_label=1)
add_lots_of_circles(phantom, labels, num2, min_size2, max_size2, 2, 2, start_label=num1+1)
print(phantom.min(), phantom.max())
print(labels.min(), labels.max())

0 2
0 300


## Make a function to count both objects and pixels

In [4]:
from re import L
from skimage.measure import label
from skimage.measure import regionprops
#labels = label(phantom)

def count_them(labels, phantom):
    objects = regionprops(labels, intensity_image=phantom)
    num1 = 0
    num2 = 0
    for obj in objects:
        if obj.mean_intensity < 1.5:
            num1 += 1
        else:
            num2 += 1
    print("num1: ", num1, "num2: ", num2)
    print("percent pixels type 1 to type 2", (np.count_nonzero(phantom == 1) / max( np.count_nonzero(phantom == 2),1) ))
    print()

count_them(labels, phantom)

num1:  199 num2:  100
percent pixels type 1 to type 2 0.6064617455687682



## Optionally view phantom in Napari

In [5]:
import napari
viewer = napari.view_image(phantom)
viewer.add_labels(phantom)

napari.manifest -> 'napari-hello' could not be imported: Cannot find module 'napari_plugins' declared in entrypoint: 'napari_plugins:napari.yaml'


<Labels layer 'phantom [1]' at 0x1ad495493d0>

## Use X-Array to create pyramids

In [6]:
import dask.array as da
from xarray_multiscale import multiscale, windowed_mode
pyramid_labels = multiscale(labels, windowed_mode, 2)
pyramid_phantom = multiscale(phantom, windowed_mode, 2)

## Check stats at each level

At each level count objects and pixels for both the windowed mode and sub-sampled image.

In [7]:
print('full res size=', labels.shape)
count_them(labels, phantom)

for i in range(1,4):
    print("====================================")
    
    
    labels_d2_mode = pyramid_labels[i]
    phantom_d2_mode = pyramid_phantom[i]

    labels_d2_subsample = labels[::2**i,::2**i]
    phantom_d2_subsample = phantom[::2**i,::2**i]
    
    print('level =', i, 'size=', labels_d2_mode.shape)
    #print(labels_d2_mode.shape)
    #print(labels_d2_subsample.shape)
    print("mode results")
    count_them(labels_d2_mode.data, phantom_d2_mode.data)
    print("subsample results")
    count_them(labels_d2_subsample, phantom_d2_subsample)

full res size= (10000, 10000)
num1:  199 num2:  100
percent pixels type 1 to type 2 0.6064617455687682

level = 1 size= (5000, 5000)
mode results
num1:  199 num2:  100
percent pixels type 1 to type 2 0.5987280436099334

subsample results
num1:  199 num2:  100
percent pixels type 1 to type 2 0.6065132954884972

level = 2 size= (2500, 2500)
mode results
num1:  194 num2:  100
percent pixels type 1 to type 2 0.5370843989769821

subsample results
num1:  199 num2:  100
percent pixels type 1 to type 2 0.615568862275449

level = 3 size= (1250, 1250)
mode results
num1:  40 num2:  92
percent pixels type 1 to type 2 0.26143790849673204

subsample results
num1:  120 num2:  100
percent pixels type 1 to type 2 0.5674418604651162

